In [1]:
#Note: when running on local computer, change urls to '/home/sarahwie/Documents/zip/subset'

In [29]:
import xml.etree.ElementTree as ET
import datetime
import os
import urlparse
import zipfile
import pickle

from xml.etree.ElementTree import Element

Set working directory to where files are:

In [30]:
os.chdir('/mnt/mypartition/Desktop2/PubMed')
#os.chdir('/home/sarahwie/Documents/zip/subset')

### Create PMIDs dictionary

In [31]:
my_list = [line.strip() for line in open('/mnt/mypartition/Desktop2/MUSC_PMIDs.txt', 'r')]
#my_list = [line.strip() for line in open('/home/sarahwie/Documents/zip/MUSC_PMIDs.txt', 'r')]

Convert list to dictionary:

In [32]:
pmids = dict(zip(my_list, range(len(my_list))))

In [33]:
print len(pmids)

38814


### Now that we have our list of indexes, extract the XML for these indexes and append to a new file:

In [ ]:
inxs_list = pickle.load(open('/mnt/mypartition/Desktop2/inxs_list.p', 'rb'))

In [34]:
print len(inxs_list)

811


In [39]:
#make new XML file
rootNew = Element('MedlineCitationSet')
children = []

In [223]:
a = datetime.datetime.now().replace(microsecond=0)

i = 0
#read in files in directory (in order)
for filename in sorted(os.listdir('/mnt/mypartition/Desktop2/PubMed/')):
    if filename.endswith(".xml.zip"):
        print filename
        #inside loop that goes through each file and unzips:
        zip_file = zipfile.ZipFile(urlparse.urljoin('/mnt/mypartition/Desktop2/PubMed/',filename))
        zip_file.extractall()
        unzip_name = zip_file.namelist()[0]
        
        xml_file = ET.parse(urlparse.urljoin('./zip/',unzip_name))
        root = xml_file.getroot()
        
        #get indexes for file:
        for inx in inxs_list[i]:
            #pull this index out of the xml
            children.append(root.getchildren()[inx])
        i = i + 1
        
        #then remove unzipped file
        os.remove(urlparse.urljoin('./zip/', unzip_name))
        
#add elements to root of tree
rootNew.extend(children)

#create tree and write to file
tree = ET.ElementTree(rootNew)
tree.write("testXMLwrite.xml")
        
pickle.dump(rootNew, open('rootNew.p', 'wb'))

b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

medline16n0189.xml.zip


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.


medline16n0190.xml.zip
medline16n0191.xml.zip
medline16n0192.xml.zip
medline16n0193.xml.zip
medline16n0194.xml.zip
medline16n0195.xml.zip
medline16n0196.xml.zip
medline16n0197.xml.zip
medline16n0198.xml.zip
medline16n0199.xml.zip
medline16n0200.xml.zip
medline16n0201.xml.zip
medline16n0202.xml.zip
medline16n0203.xml.zip
medline16n0204.xml.zip
medline16n0205.xml.zip
medline16n0206.xml.zip
medline16n0207.xml.zip
medline16n0208.xml.zip
medline16n0209.xml.zip
medline16n0210.xml.zip
medline16n0211.xml.zip
medline16n0212.xml.zip
medline16n0213.xml.zip
medline16n0214.xml.zip
medline16n0215.xml.zip
medline16n0216.xml.zip
medline16n0217.xml.zip
medline16n0218.xml.zip
medline16n0219.xml.zip
medline16n0220.xml.zip
medline16n0221.xml.zip
medline16n0222.xml.zip
medline16n0223.xml.zip
medline16n0224.xml.zip
medline16n0225.xml.zip
medline16n0226.xml.zip
medline16n0227.xml.zip
medline16n0228.xml.zip
medline16n0229.xml.zip
medline16n0230.xml.zip
medline16n0231.xml.zip
medline16n0232.xml.zip
medline16n0

### Testing that this process worked

In [44]:
rootNew = tree.getroot()
#OR
#rootNew = pickle.load(open('/mnt/mypartition/Desktop2/rootNew.p', 'rb'))

In [45]:
# how many PMID's made it into the subset file?
print len(rootNew.findall('MedlineCitation'))
print len(rootNew.getchildren())

64880


/home/nlp-vm/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  if __name__ == '__main__':


In [23]:
#make sure pmids code has been freshly run and no elements have been popped
print len(pmids)

38814


In [25]:
pmids2 = pmids.copy()
print len(pmids2)
i = 0
j = 0
k = 0
for record in rootNew.findall('MedlineCitation'):
    pmid = record.find('PMID').text
    if pmid in pmids2:
        pmids2.pop(pmid)
        k = k + 1
        #print i
    else:
        j = j + 1
        #print i
    i = i + 1
    
pickle.dump(pmids2, open('pmidsLeft.p', 'wb'))

38814


In [27]:
#number of records in pmids dictionary (should be same as i value)
print k
#number not***should be 0
print j
#total number of records (expecting 32408)
print i

471
31937
32408


In [28]:
#How many PMIDs weren't in records (expecting ~6405)
print len(pmids2)

38343


### Extra testing stuff

In [ ]:
# getting pmid duplicates
i = 0
seen = set()
for record in rootNew.findall('MedlineCitation'):    
    #get pmid
    pmid = record.find('PMID').text
    if pmid not in seen:
        seen.add(x)
    else:
        print i
    i = i + 1


rootNew.remove(record)

In [47]:
#compare pmids dict now with original
shared_items = set(pmids.items()) & set(pmids2.items())
print 38814-len(shared_items)

30


In [173]:
#Checking that the dictionary was properly created by sorting
import operator
sorted_pmids = sorted(pmids.items(), key=operator.itemgetter(1))
#print sorted_pmids